In [ ]:
!pip install transformers
!pip install torch
!pip install datasets
from transformers import BertForQuestionAnswering, BertTokenizer, BertConfig
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
import torch
from tqdm import tqdm

# Load the pre-trained model and tokenizer
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased').to('cuda')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Fine-tuning settings
learning_rate = 5e-5
batch_size = 8
num_epochs = 3

# Define optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Fine-tune the model on SQuAD dataset
# You should replace this with your own SQuAD dataset
from datasets import load_dataset
dataset = load_dataset('squad')

train_dataset = dataset['train']
eval_dataset = dataset['validation']

# Tokenize the questions, contexts, and answers
tokenized_inputs = tokenizer(train_dataset['question'], train_dataset['context'], padding=True, truncation=True, return_tensors='pt'

# Create start and end positions
start_positions = train_dataset['start_positions']
end_positions = train_dataset['end_positions']

start_positions_tensor = torch.tensor(start_positions)
end_positions_tensor = torch.tensor(end_positions)

# Create a TensorDataset
dataset = TensorDataset(tokenized_inputs['input_ids'], tokenized_inputs['attention_mask'], start_positions_tensor, end_positions_tensor)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Fine-tune the model
model.train()
for epoch in range(num_epochs):
    for batch in tqdm(dataloader, desc="Epoch {}".format(epoch + 1)):
        optimizer.zero_grad()
        input_ids, attention_mask, start_positions, end_positions = batch
        outputs = model(input_ids.to('cuda'), attention_mask=attention_mask.to('cuda'), start_positions=start_positions.to('cuda'), end_positions=end_positions.to('cuda'))
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the fine-tuned model
model.save_pretrained('fine_tuned_model')

# Inference using the fine-tuned model
question = "Who wrote Romeo and Juliet?"
context = "Romeo and Juliet was written by William Shakespeare."
inputs = tokenizer(question, context, return_tensors='pt')
inputs = {k: v.to('cuda') for k, v in inputs.items()}
start_logits, end_logits = model(**inputs).values()

answer_start = torch.argmax(start_logits)
answer_end = torch.argmax(end_logits) + 1
answer_tokens = inputs['input_ids'][0][answer_start:answer_end].tolist()

answer = tokenizer.decode(answer_tokens)

print("Question:", question)
print("Predicted Answer:", answer)
